formatting tips: https://gist.github.com/nikhilnayyar002/7a35e653d3d590e317c829243e73b110
or chatgpt idk

By: Andrew Yang, Megan Yu, and Frank Deegbe

# Part 1: Data Curation
### NFL-Data
Data extracted from Fantasy.NFL.com divided by season and player position, along with weekly numbers for the 2021 season.

The extract goes back to the 2015 season as that is the first available season with a full dataset for all positions.

Point total and player-ranks are specific for the MML leagues point system but all outer data are raw and can be used for other calculations and simulations.

All data are available in comma-separated csv files or row-divided JSON.

During this project, we will be using Python language, and we use tools such as [VS Code](https://code.visualstudio.com/) and [Jupiter Notebook](https://jupyter.org/) to develop this project.

Just like any other Python project, we need to import some libraries. Here are some of the libraries we will be using throughout this tutorial.

<center><b>Imports</b></center>


In [4]:
# !pip install numpy pandas seaborn matplotlib folium
import os
import folium
import warnings
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# this is just an example idk if we gonna need em

In [9]:

years = ["2020", "2021", "2022", "2023"]
# array of dataframes containing RB data for years 2020-2023
rb_dfs = []
for year in years:
    rb_url = "https://raw.githubusercontent.com/hvpkod/NFL-Data/main/NFL-data-Players/"+ year +"/RB_season.csv"
    rb_dfs.append(pd.read_csv(rb_url))

rb_dfs[0]

,PlayerName,PlayerId,Pos,Team,PassingYDS,PassingTD,PassingInt,RushingYDS,RushingTD,ReceivingRec,...,TouchReceptions,Touches,TargetsReceptions,Targets,ReceptionPercentage,RzTarget,RzTouch,RzG2G,Rank,TotalPoints
0,Alvin Kamara,2558019,RB,NO,NaN,NaN,NaN,932.0,16.0,83.0,...,83.0,270.0,83.0,107.0,78.0,17.0,57.0,29.0,1,340.7
1,Derrick Henry,2556075,RB,TEN,NaN,NaN,NaN,2027.0,17.0,19.0,...,19.0,397.0,19.0,31.0,61.0,1.0,65.0,30.0,2,322.6
2,Dalvin Cook,2557991,RB,MIN,NaN,NaN,NaN,1557.0,16.0,44.0,...,44.0,356.0,44.0,54.0,81.0,2.0,66.0,32.0,3,312.8
3,David Montgomery,2562502,RB,CHI,NaN,NaN,NaN,1070.0,8.0,54.0,...,54.0,301.0,54.0,68.0,79.0,12.0,53.0,23.0,4,239.5
4,Jonathan Taylor,2564148,RB,IND,NaN,NaN,NaN,1169.0,11.0,36.0,...,36.0,268.0,36.0,39.0,92.0,5.0,52.0,19.0,5,236.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,Damien Williams,2550512,RB,CHI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,229,0.0
229,Javonte Williams,2566049,RB,DEN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,230,0.0
230,Pooka Williams Jr.,2566788,RB,CIN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,231,0.0
231,Ty'Son Williams,2565542,RB,BAL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,232,0.0
